<a href="https://colab.research.google.com/github/sandeeppalakkal/ML_Datascience_Practice/blob/master/TensorFlow_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Practice

Reference: Introduction to TensorFlow, Google Cloud, [Coursera](https://www.coursera.org/learn/intro-tensorflow/home/welcome)

In [0]:
import tensorflow as tf

### Defining a DAG

In [2]:
a = tf.constant([1,3,4])
b = tf.constant([4,3,2])
c = tf.add(a,b)
print(c)

Tensor("Add:0", shape=(3,), dtype=int32)


### Running a Session

In [3]:
with tf.Session() as sess:
  result = sess.run(c)
  print(result)

[5 6 6]


In [4]:
print(result)
print(type(result))

[5 6 6]
<class 'numpy.ndarray'>


### Different Ways of Running a Session & Printing Results

In [0]:
x = tf.constant([3,5,7])
y = tf.constant([1,2,3])
z1 = tf.add(x,y)
z2 = x * y
z3 = z2 - z1

In [6]:
print(z2)

Tensor("mul:0", shape=(3,), dtype=int32)


In [7]:
with tf.Session() as sess:
  print(sess.run(z3))

[-1  3 11]


In [8]:
with tf.Session() as sess:
  print(z3.eval())

[-1  3 11]


In [9]:
with tf.Session() as sess:
  a1,a2 = sess.run([z1,z3])
  print(a1)
  print(a2)

[ 4  7 10]
[-1  3 11]


In [10]:
with tf.Session() as sess:
  print(sess.run([z1]))

[array([ 4,  7, 10], dtype=int32)]


### TensorFlow Eager Evaluation (Non-Lazy)

In [0]:
# Nead to restart the kernel to make this code working due to TF Eager
import tensorflow as tf
from tensorflow.contrib.eager.python import tfe

In [12]:
tfe.enable_eager_execution()
x = tf.constant([3,5,7])
y = tf.constant([1,2,3])
x - y #printed due to Eager running

ValueError: ignored

### Visualizing Graph

In [0]:
import tensorflow as tf

In [0]:
# Add 'name = "x"' to label nodes in TensorBoard visualization
x = tf.constant([3,5,7],name="x")
y = tf.constant([1,2,3],name="y")
z1 = tf.add(x,y,name="z1")
z2 = x * y
z3 = z2 - z1

In [0]:
with tf.Session() as sess:
  # Write the session graph to summary directory
  with tf.summary.FileWriter('summaries',sess.graph) as writer:
    a1,a2 = sess.run([z1,z3])

In [0]:
!ls summaries

In [0]:
# Need ngrok tunnelling to run TensorBoard from Colab
# Ref: https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/
from google.datalab.ml import TensorBoard
TensorBoard().start('./summaries')

### Tensors in TF

A TF graph consists of tensors and operations.

In [0]:
x = tf.constant([[[1,2,3],[4,5,6]]])
print(x)

In [0]:
print(x.shape)

In [0]:
x = tf.constant(10)
print(x.shape)
x = tf.constant([10])
print(x.shape)
x = tf.constant([10,10])
print(x.shape)
x = tf.constant([[10,10],[10,10]])
print(x.shape)
x = tf.constant([[[10,10,10],[10,10,10]]])
print(x.shape)
x = tf.constant([[[10,10,10],[10,10,10],[10,10,10]]])
print(x.shape)
x = tf.constant([[[10,10,10],[10,10,10],[10,10,10]],[[10,10,10],[10,10,10],[10,10,10]]])
print(x.shape)

In [0]:
x = tf.constant(10)
print(x.shape)
x = tf.stack([x,x])
print(x.shape)
x = tf.stack([x,x,x])
print(x.shape)
x = tf.stack([x,x,x,x])
print(x.shape)
x = tf.stack([x,x])
print(x.shape)

In [0]:
x = tf.constant([10,10])
print(x.shape)

### Slicing of a Tensor

In [0]:
x = tf.constant([[3,5,7],[4,6,8]])
y = x[:,1]
z = x[1,:]
with tf.Session() as sess:
  a1,a2,a3 = sess.run([x,y,z])
  print("x = ", a1)
  print("y = ",a2)
  print("z = ",a3)

### Reshaping a Tensor

In [0]:
x = tf.constant([[3,5,7],[4,6,8]])
y = tf.reshape(x,[3,2])
z = tf.reshape(x,[3,2])[2,:]
print(x.shape,y.shape)

In [0]:
with tf.Session() as sess:
  print('x = ', x.eval())
  print('y = ', y.eval())
  print('z = ', z.eval())

### Variables

In [0]:
def forward_pass(w,x):
  return tf.matmul(w,x)

def train_loop(x,n_iter=10):
  # Defien scope for w and tell TF to reuse the w from previous definition
  with tf.variable_scope("model",reuse=tf.AUTO_REUSE):
    # Define the variable, w
    w = tf.get_variable("weights",
                       shape=(1,2), #shape 1x2
                       initializer=tf.truncated_normal_initializer(), #init
                       trainable=True) #trainable or not
    preds = []
    for k in range(n_iter):
      preds.append(forward_pass(w,x))
      w = w + 0.1
    return preds

In [0]:
with tf.Session() as sess:
  #preds = train_loop(tf.constant([[3.2,5.1,7.2],[4.3,6.2,8.3]])) #2x3 matrix
  preds = train_loop(tf.constant([[1.,1.,1.,2.],[1.,1.,1.,2.]])) #2x4 matrix
  # initialization of all the variables together
  tf.global_variables_initializer().run()
  for i in range(len(preds)):
    print("{}:{}".format(i,preds[i].eval()))

### Placeholders

Allow you to feed in values, such as by reading from a text file.

In [0]:
a = tf.placeholder("float",None)
b = a * 4
print(a)
with tf.Session() as sess:
  print(sess.run(b,feed_dict={a:[1,2,4]}))

### TF Estimators

In [0]:
import tensorflow as tf
import shutil

In [0]:
shutil.rmtree("outdir",ignore_errors=True)

In [0]:
# Training data input function
def train_input_fn():
  features = {"sq_footage": [ 1000,    2000,     3000,  1000,  2000,  3000 ],
              "type"      : ["house", "house", "house", "apt", "apt", "apt"]}
  labels   =                [  500,     1000,    1500,   700,  1300,  1900 ]
  return features, labels

In [0]:
# tf model input
featcols = [
    tf.feature_column.numeric_column("sq_footage"),
    tf.feature_column.categorical_column_with_vocabulary_list("type",["house","apt"])    
]

In [17]:
# Model
model = tf.estimator.LinearRegressor(featcols,"outdir")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'outdir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f428d77b2e8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [18]:
# training
model.train(train_input_fn,steps=2000)

Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into outdir/model.ckpt.
INFO:tensorflow:loss = 9290000.0, step = 0
INFO:tensorflow:global_step/sec: 267.819
INFO:tensorflow:loss = 144242.62, step = 100 (0.379 sec)
INFO:tensorflow:global_step/sec: 318.776
INFO:tensorflow:loss = 142023.22, step = 200 (0.314 sec)
INFO:tensorflow:global_step/sec: 298.535
INFO:tensorflow:loss = 140171.67, step = 300 (0.335 sec)
INFO:tensorflow:global_step/sec: 287.278
INFO:tensorflow:loss = 138553.88, step = 400 (0.344 sec)
INFO:tensorflow:global_step/sec: 292.398
INFO:tensorflow:loss = 137101.47, step = 500 (0.342 se

In [0]:
# Prediction data input function
def predict_input_fn():
  features = {"sq_footage": [ 1500,    1500,   2500,   2500 ],
              "type"      : ["house",  "apt", "house", "apt"]}
  return features

In [0]:
# prediction model
predictions = model.predict(predict_input_fn)

In [21]:
# Predict
print(next(predictions))
print(next(predictions))
print(next(predictions))
print(next(predictions))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from outdir/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([844.07806], dtype=float32)}
{'predictions': array([871.82715], dtype=float32)}
{'predictions': array([1414.5388], dtype=float32)}
{'predictions': array([1442.2878], dtype=float32)}
